#  Práctica Obligatoria - Parte IV - Visualización

***<p style="text-align:center;">Aprendizaje Automático II</p>***
***<p style="text-align:center;">Visualización</p>***

En esta parte, cargarás un modelo CNN entrenado y visualizarás los parámetros y sus activaciones.

### Evaluación - 1.5/10 puntos

Puntuación de cada parte sobre el total de la práctica:
- **[Ejercicio 1]** 1.5 puntos.

In [ ]:
!unzip data.zip

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import transforms

from gts_dataset import GTS

## Importa las funciones que has creado desde `utils.py`:

In [3]:
from utils import count_trainable_params, train, evaluate, train_and_evaluate, save_full_model, load_full_model, plot_loss_accuracy, plot_confusion_matrix, plot_error_per_class

## Carga el dataset GTS  y crea los dataloaders:

In [4]:
# Definir las transformaciones y conversión a tensor
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = GTS(csv_file='train.csv', root_dir='./data', transform=transform)
valid_dataset = GTS(csv_file='valid.csv', root_dir='./data', transform=transform)
test_dataset = GTS(csv_file='test.csv', root_dir='./data', transform=transform)

device = 'cpu'
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# Carga el modelo `CNN` entrenado (`cnn.pth`):

In [ ]:
from models import CNN

model_cnn = load_full_model('cnn.pth')  

model_cnn

Modelo cargado desde cnn.pth


/content/utils.py:154: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(file_name)


CNN(
  (block1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=576, out_features=43, bias=True)
)

## **[Ejercicio 1]** Visualiza los filtros de la primera capa convolucional y sus activaciones para una imagen dada:

Para acceder a los filtros, puedes hacer uso de la notación por punto:

`model_cnn.conv1.weight.data`

Para visualizar las activaciones, deberás ejecutar cada función del bloque convolucional que necesites (conv --> bn --> relu).

In [ ]:
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt

def plot_filter_and_activation(model, image, device='cpu'):
    """
    Visualiza la imagen original, un filtro y su activación correspondiente.
    """
    model.eval()  # Asegurar modo evaluación

    with torch.no_grad():
       
        # Calcular activaciones de la primera capa
        block1 = model.block1[0]
        activations = block1(image).squeeze(0).cpu()

   
    # Procesar filtros y activaciones para visualización
    filters = block1.weight.data    # Filtros de la primera capa

    image_np = image.permute(1, 2, 0).cpu().detach().numpy()  # Imagen original en formato HWC para visualizar

    # Asegurar valores entre 0 y 1 para la imagen original
    image_np = (image_np - image_np.min()) / (image_np.max() - image_np.min())

    # Unificar el número de elementos para el slider
    num_elements = min(filters.shape[0], activations.shape[0])

    def plot_image_filter_activation(index):
        """
        Visualiza la imagen original, un filtro y su activación correspondiente.
        """
        # Configurar la figura
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # Mostrar imagen original
        axs[0].imshow(image_np)
        axs[0].set_title("Imagen Original")
        axs[0].axis('off')

        # Mostrar filtro
        filter_img = filters[index, :, :, :].permute(1, 2, 0).detach().numpy()
        filter_img = (filter_img - filter_img.min()) / (filter_img.max() - filter_img.min())
        axs[1].imshow(filter_img)
        axs[1].set_title(f"Filtro {index+1}")
        axs[1].axis('off')

        # Mostrar activación
        activation_img = activations[index, :, :].detach().numpy()
        axs[2].imshow(activation_img, cmap='gray')
        axs[2].set_title(f"Activación {index+1}")
        axs[2].axis('off')

        plt.tight_layout()
        plt.show()

    # Crear un slider interactivo
    interact(plot_image_filter_activation, index=IntSlider(min=0, max=num_elements-1, step=1, description="Índice"))

# Ejemplo: Usar la visualización unificada con imagen original
example_image, _ = test_dataset[0]  # Seleccionar una imagen del conjunto de prueba
plot_filter_and_activation(model_cnn, example_image)

# Al descargar el archivo, las imagenes no se pueden representar, pero al ejecutarlo de nuevo funciona

interactive(children=(IntSlider(value=0, description='Índice', max=15), Output()), _dom_classes=('widget-inter…

In [ ]:
def shift_image_and_visualize_activations(model, image, device='cpu'):
    """
    Visualiza cómo las activaciones cambian al desplazar la imagen horizontal y verticalmente.
    """
    model.eval()  # Asegurar modo evaluación
    with torch.no_grad():
       
        # Calcular activaciones de la primera capa
        block1 = model.block1[0]
        activations = block1(image).squeeze(0).cpu()

    
    # Procesar filtros y activaciones para visualización
    filters = block1.weight.data  # Filtros de la primera capa

    image_np = image.permute(1, 2, 0).cpu().numpy()  # Imagen original en formato HWC para visualizar

    # Asegurar valores entre 0 y 1 para la imagen original
    image_np = (image_np - image_np.min()) / (image_np.max() - image_np.min())

    def shift_and_plot_activation(horizontal_shift, vertical_shift):
        """
        Traslada la imagen en las direcciones horizontal y vertical, y visualiza el cambio en las activaciones.
        """
        # Desplazar la imagen horizontal y verticalmente
        shifted_image = np.roll(image_np, horizontal_shift, axis=1)  # Desplazamiento horizontal
        shifted_image = np.roll(shifted_image, vertical_shift, axis=0)  # Desplazamiento vertical

        # Configurar la figura
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # Mostrar imagen original
        axs[0].imshow(image_np)
        axs[0].set_title("Imagen Original")
        axs[0].axis('off')

        # Mostrar imagen desplazada
        axs[1].imshow(shifted_image)
        axs[1].set_title(f"Imagen Desplazada (Horizontal: {horizontal_shift}, Vertical: {vertical_shift})")
        axs[1].axis('off')

        # Calcular activaciones para la imagen desplazada
        shifted_image_tensor = torch.tensor(shifted_image).permute(2, 0, 1).unsqueeze(0).float()
        shifted_activations = block1(shifted_image_tensor).squeeze(0).cpu()

        # Mostrar activación de la imagen desplazada
        activation_img = shifted_activations[0, :, :].detach().numpy()
        axs[2].imshow(activation_img, cmap='inferno')
        axs[2].set_title(f"Activación Después Desplazamiento")
        axs[2].axis('off')

        plt.tight_layout()
        plt.show()

    # Crear sliders interactivos para el desplazamiento horizontal y vertical
    max_shift = 10  # Máximo desplazamiento en píxeles
    interact(shift_and_plot_activation,
             horizontal_shift=IntSlider(min=-max_shift, max=max_shift, step=1, description="Desp. Hor"),
             vertical_shift=IntSlider(min=-max_shift, max=max_shift, step=1, description="Desp. Vert"))

example_image, _ = test_dataset[0]  # Seleccionar una imagen del conjunto de prueba
shift_image_and_visualize_activations(model_cnn, example_image)

# Al descargar el archivo, las imagenes no se pueden representar, pero al ejecutarlo de nuevo funciona

interactive(children=(IntSlider(value=0, description='Desp. Hor', max=10, min=-10), IntSlider(value=0, descrip…